In [1]:
import pandas as pd
import os

cwd = os.getcwd()
os.chdir(cwd+'/../../data/output csv files')

ratings = pd.read_csv('finalratings.csv')
finalbooks = pd.read_csv('finalbooks.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

os.chdir(cwd)

In [2]:
# Sort all books based on Popularity

# groupbooks by newbookid and get sum of ratings_count and average_ratings columns and sort them based on ratings_count in descending order and reset index
popular = finalbooks.groupby('newbookid')['ratings_count','average_rating'].sum().sort_values(by = 'ratings_count',ascending=False).reset_index()
popular

<ipython-input-2-633b7d386aea>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  popular = finalbooks.groupby('newbookid')['ratings_count','average_rating'].sum().sort_values(by = 'ratings_count',ascending=False).reset_index()


,newbookid,ratings_count,average_rating
0,1,4780653,4.34
1,2,4602479,4.44
2,3,3866839,3.57
3,4,3198671,4.25
4,5,2683664,3.89
...,...,...,...
7995,7322,4886,3.89
7996,7153,4842,3.93
7997,7501,4528,4.29
7998,7807,4281,3.73


In [3]:
useronly = ratings['newuser_id'].unique()
bookonly = popular['newbookid'].tolist()
top_predictions = pd.DataFrame({'newuser_id':pd.Series([], dtype='int'), 'newbookid':pd.Series([], dtype='int')})

# for i in range(len(useronly)):
#     all_books = bookonly
#     read_books = train.loc[ratings.newuser_id == i+1]['newbookid'].tolist()
#     rem_books = [i for i in bookonly if i not in read_books]
#     top_10=rem_books[:10]
#     predictions.append(top_10)
#     if (i+1)%1000 == 0: print("done: ", i+1)
        
for i in range(len(useronly)):
    all_books = bookonly
    top_10=[]
    read_books = train.loc[ratings.newuser_id == i+1]['newbookid'].tolist()
    count=0
    k=0
    while(count<10):
        if all_books[k] not in read_books:
            top_predictions = top_predictions.append({'newuser_id': i+1, 'newbookid': all_books[k]}, ignore_index = True) 
            count=count+1
        k=k+1
    if (i+1)%1000 == 0: print("done: ", i+1)

top_predictions

done:  1000
done:  2000
done:  3000
done:  4000
done:  5000
done:  6000
done:  7000
done:  8000
done:  9000
done:  10000
done:  11000
done:  12000
done:  13000
done:  14000
done:  15000


,newuser_id,newbookid
0,1,2
1,1,3
2,1,5
3,1,8
4,1,9
...,...,...
149995,15000,6
149996,15000,7
149997,15000,8
149998,15000,10


In [4]:
top_predictions = top_predictions.merge(popular[['newbookid', 'ratings_count', 'average_rating']], how='left', on='newbookid')
top_predictions.to_csv("top_10_predictions.csv", index=False)

In [5]:
# predict ratings for a book that might be given by user in test data

import numpy as np

# merge test dataframe and popular dataframe
poprank = test.merge(popular,on = 'newbookid')

# pred column gives average_rating rounded to integer value
poprank['pred']= np.round(poprank['average_rating'])

# ratingYN column assigns 1 if rating>=4 and 0 if rating<4
poprank['ratingYN']= [1 if x>= 4 else 0 for x in poprank['rating']]


poprank = poprank.sort_values(by=['newuser_id', 'ratings_count'], ascending=False)
poprank

,rating,newbookid,newuser_id,ratings_count,average_rating,pred,ratingYN
139520,4,3,15000,3866839,3.57,4.0,1
143201,5,23,15000,1779331,4.37,4.0,1
149103,5,42,15000,1257121,4.04,4.0,1
77754,4,70,15000,813439,4.30,4.0,1
19823,4,89,15000,628637,4.25,4.0,1
...,...,...,...,...,...,...,...
145357,3,660,1,129811,3.55,4.0,0
153191,3,903,1,95620,3.62,4.0,0
334789,4,1256,1,82458,3.94,4.0,1
55873,3,1294,1,65397,3.69,4.0,0


In [6]:
def rmse(y,h):
    """RMSE
    Args:
        y: real y
        h: predicted y
    Returns:
        RMSE
    """
    a = y-h

    return np.sqrt(sum(a**2)/len(a))

In [7]:
# root

print('(1) Popularity Model RMSE: ', np.round(rmse(poprank['rating'],poprank['average_rating']), decimals=3))

(1) Popularity Model RMSE:  0.968
